# Finding Confidence Score for Entities

For Customized Entities

In [1]:
# for Python 2: use print only as a function
from __future__ import print_function

In [2]:
import spacy
from spacy.pipeline import EntityRuler
from spacy.matcher import Matcher
from spacy.tokens import Span
from spacy import displacy
import sys
from collections import defaultdict
nlp = spacy.load('en_core_web_sm')

In [3]:
from spacy.scorer import Scorer

scorer = Scorer()

In [4]:
flowers = ["rose", "tulip", "african daisy"]
animals = ["cat", "dog", "artic fox"]

In [5]:
rulerAll = EntityRuler(nlp, overwrite_ents=True)

In [6]:
for f in flowers:
    rulerAll.add_patterns([{"label": "flower", "pattern": f}])
for a in animals:
    rulerAll.add_patterns([{"label": "animal", "pattern": a}])

In [7]:
print(nlp.pipe_names)

['tagger', 'parser', 'ner']


In [8]:
# in this case, we shall use same ruler for all classes
rulerAll.name = 'rulerAll'

In [9]:
nlp.add_pipe(rulerAll)

In [10]:
print(nlp.pipe_names)

['tagger', 'parser', 'ner', 'rulerAll']


In [11]:
text="cat, fox, dog, wolf and artic fox are one class,and plant, african daisy, rose ,tulip, gardenias are other class"

In [12]:
doc=nlp(text)

In [13]:
threshold = 0.2
beams = nlp.entity.beam_parse([ doc ], beam_width = 16, beam_density = 0.0001)

In [14]:
beams

In [15]:
entity_scores = defaultdict(float)
for beam in beams:
    for score, ents in nlp.entity.moves.get_beam_parses(beam):
        for start, end, label in ents:
            entity_scores[(start, end, label)] += score

print ('Entities and scores (detected with beam search)')
for key in entity_scores:
    start, end, label = key
    score = entity_scores[key]
    if ( score > threshold):
        print ('Label: {}, Text: {}, Score: {}'.format(label, doc[start:end], score))

Entities and scores (detected with beam search)
Label: animal, Text: cat, Score: 0.9999999999999996
Label: PERSON, Text: fox, Score: 0.9999999999999996
Label: animal, Text: dog, Score: 0.9999999999999996
Label: PERSON, Text: wolf, Score: 0.9999999999999996
Label: animal, Text: artic fox, Score: 0.9999999999999996
Label: CARDINAL, Text: one, Score: 0.9999999999999996
Label: flower, Text: african daisy, Score: 0.9999999999999996
Label: flower, Text: rose, Score: 0.9999999999999996
Label: flower, Text: tulip, Score: 0.9999999999999996
Label: PERSON, Text: gardenias, Score: 0.6247260804783323
